In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://mikescamera.com/used-gear#used-gear-california"
page = requests.get(url)

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
ca_used_gear_url = soup.find('iframe', title='California used gear table').attrs['src']
ca_used_gear_url

'https://docs.google.com/spreadsheets/d/e/2PACX-1vSMKmlrvBlo_xuCv1wfZXdNLyRg3vpV0FW_Lz6DxsmgBmmwS8IxDQSiEV-tOqKnfwtp-KOyuwjDYW49/pubhtml?gid=2102305869&single=true&widget=false&headers=false&chrome=false'

In [5]:
gear_list = requests.get(ca_used_gear_url)

In [6]:
gear_soup = BeautifulSoup(gear_list.content, 'html.parser')

In [7]:
gear = gear_soup.table

In [8]:
for th in gear("th"):
    th.decompose()

In [9]:
# for a in gear("a"):
#     a.decompose()

In [10]:
gs = gear.find_all('tr')[1:]

In [11]:
values = []
for string in gs[3].stripped_strings:
   values.append(string)

In [ ]:
def cleanup(i: str) -> str|int:
    if '$' in i:
        return int(i.replace('$ ', '').replace('.00', ''))
    else:
        try:
            return int(i)
        except ValueError:
            return i

In [12]:
def item_dict(tag):
    headers = ['ref', 'name', 'price', 'loc', 'num'] 
    values = []
    for string in tag.stripped_strings:
        values.append(cleanup(string))

    return dict(zip(headers, values))

In [13]:
mega_list = []
for tag in gs:
    mega_list.append(item_dict(tag))

In [14]:
import json
with open('mike.json', 'w') as f:
    json.dump(mega_list, f, indent = 2)